# Install

In [ ]:
pip install image # install's PIL, the python image library
# install portaudio is required by pyaudio (http://portaudio.com/docs/v19-doxydocs/tutorial_start.html)
pip install pyaudio # for recording within python. used for audioSearch module
pip install pygame # for realtime MIDI performance in midi.realtime module
# install muscore for viewing and editing music notation (http://www.musescore.org)
# install lilypond for displaying musical scores (http://lilypond.org/) 

# Import

In [372]:
# directory, files management, etc.
import os
from os.path import isfile, join
import glob
from StringIO import StringIO
import time
import warnings

# arrays, dataframes
import numpy as np
import pandas as pd

# utilities
import random as rand
import time
import matplotlib.pyplot as plt
%matplotlib inline

# audio
import wave
#import pyaudio

# MIR
import essentia
import madmom as mad
import music21 as m21

# neural nets

# Globals

In [561]:
# pitches, notes
PITCH_CLASSES = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
NUM_PITCH_CLASS = len(PITCH_CLASSES)
NUM_MIDI_PITCH = 127               # range of audible sounds
RANGE_PIANO_PITCH = range(21,109)  # 88 notes (12 notes * 7.25 octave scale), midi 21:108
NUM_PIANO_PITCH = len(RANGE_PIANO_PITCH)

# chords
NUM_COM_CHORD_OCTAVE = 5
#NUM_ALL_CHORD_PITCH = NUM_PIANO_PITCH # midi 21:108 inclusive. 7.25 octave piano scale
#RANGE_COM_CORD_PITCH = range(36,96) # midi 36:95 inclusive, 5 octave piano scale
#NUM_COM_CHORD_PITCH = len(RANGE_COM_CHORD_PITCH)
NUM_MAJOR_MINOR = 2
CHORD_KEY_SETS = 1 # only 3-key chords 

# loudness
RANGE_MEZZO_FORTE = range(60, 69)
RANGE_PIANO_FORTE = range(32, 97)

# classes
NUM_NOT_TGT_CLASS = 1
NOT_TGT_LABEL = ["not target"]
NUM_PITCH_CLASSES = NUM_PIANO_PITCH + NUM_NOT_TGT_CLASS
NUM_CHORD_CLASSES = (NUM_PITCH_CLASS * NUM_COM_CHORD_OCTAVE * NUM_MAJOR_MINOR * CHORD_KEY_SETS) + NUM_NOT_TGT_CLASS

#PITCH_CLASSES = RANGE_PIANO_PITCH
#PITCH_CLASSES = NOT_TGT_LABEL + PITCH_CLASSES

CHORD_CLASSES = []
for k in range(3,8):
    for i in NOTES:
        for j in ["major", "minor"]:
            CHORD_CLASSES.append(str(i+str(k)+"-"+j+" triad"))
CHORD_CLASSES = NOT_TGT_LABEL + CHORD_CLASSES

# to standardize feature extract algos
NUM_SAMPLES = 22050 # 44100
NUM_FRAMES = 1024 # 2048
NUM_HOPS = 512 # 441
NUM_BANDS = 24 # 48

# other 
dat_dir = 'data/maps/'
WINDOW = 0.025

# Pre-Process Data

# Acoustic Model

## 1. Build Train and Test sets

In [769]:
'''for testing performance of madmom crf/cnn chord identification modules. 
grabs directory files. tests both algorithms against text on/offset file. '''

def process_wavs(inWavDir, outTxtDir):
    '''
    reads wav, writes note, cnn chord and dnn chord files to target dir
    if tgtWavDir provided, appends chord, note and wav files incrementing. saves wav to target dir. 
    '''
    
    inWavs = glob.glob(inWavDir + '/*/*.wav')
    print(len(inWavs))
    wavs = []; begTm = 0; noteCtr = 0; chordCtr = 0
    
    for i in range(len(inWavs)):
        if i % 5 == 0: print("processing input file #", i)
        
        # process X: chord and note recognition from wave
        dnm = os.path.dirname(inWavs[i])
        fnm, _ = os.path.splitext(os.path.basename(inWavs[i]))
        inNm = dnm + '/' + fnm + '.txt'
        outNm = outTxtDir + '/' + fnm
        
        dnn_chord_rec(inWavs[i], savTo = outNm + '.chords.dnn.txt' )
        #cnn_chord_rec(inWavs[i], savTo = outNm + '.chords.cnn.txt'  )
        rnn_note_and_chord_rec("single", inWavs[i], outNm) # outNm + '.notes.txt'
        
        # process Y: corresponding y values from text files
        N, C = txt_to_y(inNm)
        
        if N is not None:
            N['Duration'] = N.OffsetTime - N.OnsetTime
            N = N[['OnsetTime', 'MidiPitch', 'Duration' ]]
            mad.features.notes.write_notes(np.array(N),
                                           outNm + '.note.y.txt',
                                           fmt=['%.3f', '%d', '%.3f'])
            
        if C is not None:
            save_chords(outNm + '.chords.y.txt', C)

#process_wavs(inWavDir = 'data/wip/tmp/',
#             outTxtDir = 'data/wip/tmp')
    
process_wavs(inWavDir = 'data/maps/AkPnBcht/UCHO/I60-68/',
             outTxtDir = 'data/wip/AkPnBcht/UCHO/I60-68/')


885
('processing input file #', 0)


/Users/mdowns/.virtualenvs/audio_2.7_env/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


Traceback (most recent call last):
  File "/Users/mdowns/.virtualenvs/audio_2.7_env/bin/PianoTranscriptor", line 123, in <module>
    main()
  File "/Users/mdowns/.virtualenvs/audio_2.7_env/bin/PianoTranscriptor", line 119, in main
    args.func(processor, **vars(args))
  File "/Users/mdowns/.virtualenvs/audio_2.7_env/lib/python2.7/site-packages/madmom/processors.py", line 518, in process_single
    processor(infile, outfile)
  File "/Users/mdowns/.virtualenvs/audio_2.7_env/lib/python2.7/site-packages/madmom/processors.py", line 119, in __call__
    return self.process(*args, **kwargs)
  File "/Users/mdowns/.virtualenvs/audio_2.7_env/lib/python2.7/site-packages/madmom/processors.py", line 499, in process
    return _process((self.out_processor, data, output))
  File "/Users/mdowns/.virtualenvs/audio_2.7_env/lib/python2.7/site-packages/madmom/processors.py", line 186, in _process
    return process_tuple[0](*process_tuple[1:])
  File "/Users/mdowns/.virtualenvs/audio_2.7_env/lib/python2

In [702]:
'''achieves 45% major/minor triad recogniation. significantly slower than dnn model'''

def cnn_chord_rec(inWav, savTo=None):
    
    '''from Filip Korzeniowski and Gerhard Widmer, “A Fully Convolutional Deep Auditory Model for 
    Musical Chord Recognition”, Proceedings of IEEE International Workshop on Machine Learning for
    Signal Processing (MLSP), 2016..'''
    
    # instantiate madmom CNNChordFeatureProcessor
    featproc = mad.features.chords.CNNChordFeatureProcessor()
    
    # create DeepChromaChordRecognitionProcessor to decode chord sequence from extracted chromas
    decode = mad.features.chords.CRFChordRecognitionProcessor()
    
    # SequentialProcessor links dcp and decode steps to transcribe chord(s)
    chordrec = mad.processors.SequentialProcessor([featproc, decode])
    
    madChords = chordrec(inWav)
    
    rtrn = to_chords(pd.DataFrame(madChords), 'madChordDF')
    # when you decide to expand the output classes:
        # 1. use the code here: https://github.com/CPJKU/madmom/blob/master/madmom/features/chords.py
        # 2. this: DeepChromaChordRecognitionProcessor() calls this: majmin_targets_to_chord_labels()
        # 3. the latter implements 25 classes (including N for no chord) using pred_to_cl(pred)
        # 4. so, the net is outputing preds for these classes. class preds are translated to 0-23 + 24 labels
        # 5. to go beyond these classes you're going to need to either:
            # a. roll-your-own net w/ more classes
            # b. cross vector using output from other algos i.e., RNN note or CNN chord... START HERE.
    
    if savTo is not None:
        save_chords(savTo, rtrn) 
    else:
        return(rtrn)

#cnn_chord_rec('data/maps/AkPnBcht/MUS/MAPS_MUS-alb_se3_AkPnBcht.wav',
#              'data/wip/AkPnBcht/MUS/MAPS_MUS-alb_se3_AkPnBcht.madChordCnn.txt')

In [767]:
'''achieves:
    1. 22% precision and 6% fscore on collection of map music
    2. 35% major/minor triad recognition, 
    so, clearly the 70% fscore numbers they're giving are on major/minor triads w/in a target range'''

def dnn_chord_rec(inWav, savTo=None):
    
    '''from Filip Korzeniowski and Gerhard Widmer, “Feature Learning for Chord Recognition: The Deep
    Chroma Extractor”, Proceedings of the 17th International Society for Music Information Retrieval
    Conference (ISMIR), 2016.'''
    
    # instantiate madmom deep chroma processor to extract chroma vectors
    dcp = mad.audio.chroma.DeepChromaProcessor()
    
    # create DeepChromaChordRecognitionProcessor to decode chord sequence from extracted chromas
    decode = mad.features.chords.DeepChromaChordRecognitionProcessor()
    
    # SequentialProcessor links dcp and decode steps to transcribe chord(s)
    chordrec = mad.processors.SequentialProcessor([dcp, decode])
    
    madChords = chordrec(inWav)
    
    rtrn = to_chords(pd.DataFrame(madChords), 'madChordDF')
    
    if savTo is not None:
        save_chords(savTo, rtrn)  
    else:
        return(rtrn)

#dnn_rslts = dnn_chord_rec('data/maps/AkPnBcht/MUS/MAPS_MUS-alb_se3_AkPnBcht.wav')

#dcc_rslts = dcc_chord_rec('data/maps/AkPnBcht/MUS/MAPS_MUS-alb_se3_AkPnBcht.wav',
#                          'data/wip/AkPnBcht/MUS/MAPS_MUS-alb_se3_AkPnBcht.madChordDcc.txt')

### may need to look at essentia or other from audiocommons

In [768]:
def rnn_note_and_chord_rec(mode, inFile, outFileBase=None):
    
    outNoteFile = outFileBase + '.notes.rnn.txt'
    outChordFile = outFileBase + '.chords.rnn.txt'
    
    # generate and save note transcription from .wav file
    if mode == "single":
        if outFileBase == None: 
            rtrn = ! PianoTranscriptor single {inFile}
            return(rtrn)
        else:
            ! PianoTranscriptor single {inFile} -o {outNoteFile}
    
    elif mode == "batch":
        if outFileBase == None:
            print("ERROR: need an outFile (really dir) when using using batch mode.")
        else:
            # assumes inFile is a list of files. assumes outFile is a directory
            for i in range(len(inFile)): 
                ! PianoTranscriptor batch {inFile[i]} -o {outNoteFile}
    
    # generate and save chord transcription from note transcription
    try:
        # MOVE THIS TO RE-FORMATTER
            # note save format is onset + midi (as used by eval).
            # note -> chord format is onset + offset + midi
        rnnNotes = pd.DataFrame(mad.features.notes.load_notes(outNoteFile)) # outNm + '.notes.rnn.txt'
        rnnNotes.insert(1, "Offset", pd.Series(np.zeros(rnnNotes.shape[0]), index=rnnNotes.index))
        rnnNotes.columns = ['OnsetTime', 'OffsetTime', 'MidiPitch']
        rnnNotes = rnnNotes.sort_values(['OnsetTime', 'MidiPitch'], # 'OffsetTime', 
                                        axis=0, ascending=True, inplace=False,
                                        kind='quicksort', na_position='last')
        rnnNotes["MidiPitch"] = rnnNotes['MidiPitch'].astype(int)
    
        # get chord preds using RNN notes
        N, C = txt_to_y(rnnNotes, "thisFile")
    
        if C is not None: 
            save_chords(outChordFile, C)
    except:
        pass
        
    # https://www.safaribooksonline.com/blog/2014/02/12/using-shell-commands-effectively-ipython/

#, 'data/maps/AkPnBcht/MUS/MAPS_MUS-bach_846_AkPnBcht.wav']
#rnn_note_and_chord_rec("single", 
#                       'data/maps/AkPnBcht/MUS/MAPS_MUS-alb_se3_AkPnBcht.wav', 
#                       'data/wip/tmp/MAPS_MUS-alb_se3_AkPnBcht')

In [775]:
def find_offset_window(inDetLsts, inDetNms, inAnnotLst):
    
    # technically should take 5% of rcds to determine best offest, window. not going to do that.    
    dly = np.round(np.arange(start=0.01, stop=0.11, step=0.01),3).tolist() + \
    np.round(np.arange(start=0.12, stop=0.22, step=0.02),3).tolist()
    # + np.round(np.arange(start=0.23, stop=0.5, step=0.03),3).tolist()
    wndw = dly
    
    evalCols = (0,8)
    rslts = []
    
    # for each prediction algorithm,...
    for h in range(len(inDetLsts)):
        
        lnCtr = 0
        rsltMtrx = pd.DataFrame(np.ndarray((len(dly)*len(wndw),7)),
                                columns=["predictor", "dly", "wndw", "prec", "recall", "fScr", "acc"])
        evalObjs = []
        
        # ...try different onset delays and...
        for i in range(len(dly)):
            
            # ...different window durations...
            for j in range(len(wndw)):
                
                # ...across the sample of prediction files...
                for k in range(len(inDetLsts[h])):
                    
                    evalObjs.append(ChordEvaluation(inDetLsts[h][k], inAnnotLst[k],
                                                    evalCols, window=wndw[j], delay=dly[i]))
                
                # summarize results across files by window and delay
                evalSum = ChordSumEvaluation(evalObjs, name=None)
                
                rsltMtrx.iloc[lnCtr,0] = inDetNms[h]
                rsltMtrx.iloc[lnCtr,1] = dly[i]
                rsltMtrx.iloc[lnCtr,2] = wndw[j]
                rsltMtrx.iloc[lnCtr,3] = evalSum.precision
                rsltMtrx.iloc[lnCtr,4] = evalSum.recall
                rsltMtrx.iloc[lnCtr,5] = evalSum.fmeasure
                rsltMtrx.iloc[lnCtr,6] = evalSum.accuracy
                lnCtr = lnCtr + 1
        
        rsltMtrx = rsltMtrx.sort_values(by='fScr', axis=0, ascending=False, na_position='last').iloc[0:10,:]
        rslts.append(rsltMtrx)
    
    return(rslts)
                
rnnDet = ['data/wip/AkPnBcht/MUS/MAPS_MUS-bach_846_AkPnBcht.chords.rnn.txt', 
          'data/wip/AkPnBcht/MUS/MAPS_MUS-schumm-6_AkPnBcht.chords.rnn.txt'] #glob.glob('data/wip/AkPnBcht/MUS/*.chords.rnn.txt')

cnnDet = ['data/wip/AkPnBcht/MUS/MAPS_MUS-bach_846_AkPnBcht.chords.cnn.txt',
         'data/wip/AkPnBcht/MUS/MAPS_MUS-schumm-6_AkPnBcht.chords.cnn.txt'] #glob.glob('data/wip/AkPnBcht/MUS/*.chords.cnn.txt')

dnnDet = ['data/wip/AkPnBcht/MUS/MAPS_MUS-bach_846_AkPnBcht.chords.dnn.txt',
         'data/wip/AkPnBcht/MUS/MAPS_MUS-schumm-6_AkPnBcht.chords.dnn.txt'] #glob.glob('data/wip/AkPnBcht/MUS/*.chords.dnn.txt')

fSet = [rnnDet, cnnDet, dnnDet]

fSetNm = ['RNN Chord Preds', 'CNN Chord Preds', 'DNN Chord Preds']

annot = ['data/wip/AkPnBcht/MUS/MAPS_MUS-bach_846_AkPnBcht.chords.y.txt',
        'data/wip/AkPnBcht/MUS/MAPS_MUS-schumm-6_AkPnBcht.chords.y.txt']    #glob.glob('data/wip/AkPnBcht/MUS/*.chords.y.txt')

rslt = find_offset_window(fSet, fSetNm, annot)

'''Looking for best delay and window, I find that by max'ing window up to 0.5 seconds, I still only achieve
0.044 and 0.033 F Scores for CNN and DNN chord recognition algos vs. 0.9 for RNN note -> chord rec. The basic
problem is: 
1. that the CNN and DNN chord recognition algos work only in a narrow range of:
    a. major/minor triads, 
    b. in mid-range keys e.g., midi 60-70. even then they only achieve 0.4-ish and 0.3-ish F Scores.
2. while the most popular, they represent a very small portion of total chords played.
3. the more accurate CNN algo is prohibitively slow, inappropriate for online processing.

It's unlikely that they will contribute meaningfully to an improvement to the RNN Chord Rec algo. So, rather
than attempt to combine records then use a GBM, time is better focused on:
1. operationalizing the RNN, then
2. introducing the language model... using index search to find song so that next note/chord expectation is known, then
3. swinging back at some point to train the algos on a broader set of keys, speed processing, etc. when you do this, 
here are the top 30 chords w/ window = 0.01 rounding:
0    C4-interval class 4  136
1         G2-major triad  135
2              A2-unison  120
3         F4-major triad  119
4   B-4-interval class 4  116
5    D4-interval class 3  115
6   E-4-interval class 4  110
7    E4-interval class 3  108
8              B2-unison  107
9    G4-interval class 4  106
10   C5-interval class 4  105
11  G#4-interval class 3  104
12   C4-interval class 3  103
13  G#3-interval class 3  100
14   B3-interval class 3   97
15             F3-unison   97
16  F#4-interval class 3   96
17  E-5-interval class 4   94
18            G#2-unison   92
19   G3-interval class 4   92
20        G4-major triad   92
21            C#3-unison   91
22             E2-unison   91
23             A3-unison   89
24  C#4-interval class 3   89
25             D2-unison   88
26   A4-interval class 3   87
27       E-4-major triad   87
28  B-3-interval class 4   85
29       E-3-major triad   84
#def combined_chord_pred():


In [859]:
# get most commonly occuring chords in a dataset

chordFiles = glob.glob('data/wip/AkPnBcht/MUS/*.chords.y.txt')

for i in range(len(chordFiles)):
    chords = load_chords(chordFiles[i])
    #print(chords)
    if i == 0:
        allChords = chords.m21PitchedCommonName
    else:
        allChords = allChords.append(chords.m21PitchedCommonName)

allChords = allChords.tolist(); print(len(allChords))
allChords.sort()
uniqueChords = list(set(allChords))
uniqueChords.sort()

counts = []
for i in range(len(uniqueChords)):
    counts.append(allChords.count(uniqueChords[i]))

print(sum(counts))

outP = zip(uniqueChords, counts)
outP.sort(key=lambda tup: tup[1], reverse=True)
print(pd.DataFrame(outP[0:30]))

13530
13530
                       0    1
0    C4-interval class 4  136
1         G2-major triad  135
2              A2-unison  120
3         F4-major triad  119
4   B-4-interval class 4  116
5    D4-interval class 3  115
6   E-4-interval class 4  110
7    E4-interval class 3  108
8              B2-unison  107
9    G4-interval class 4  106
10   C5-interval class 4  105
11  G#4-interval class 3  104
12   C4-interval class 3  103
13  G#3-interval class 3  100
14   B3-interval class 3   97
15             F3-unison   97
16  F#4-interval class 3   96
17  E-5-interval class 4   94
18            G#2-unison   92
19   G3-interval class 4   92
20        G4-major triad   92
21            C#3-unison   91
22             E2-unison   91
23             A3-unison   89
24  C#4-interval class 3   89
25             D2-unison   88
26   A4-interval class 3   87
27       E-4-major triad   87
28  B-3-interval class 4   85
29       E-3-major triad   84


In [851]:
# chord rec notes:
'''http://www.audiocommons.org/assets/files/AC-WP4-UPF-D4.1%20Report%20on%20the%20analysis%20and%20compilation%20of%20state-of-the-art%20methods%20for%20the%20automatic%20annotation%20of%20music%20pieces%20and%20music%20samples.pdf
Audio Commons, deliverable d4: Report on the analysis and compilation of state-of-the-art methods 
for the automatic annotation of music pieces and music samples:

Recent work utilize deep learning to learn alternative features for replacing chroma features [Zhou15]. In
their work, authors investigate two types of architectures for the neural net, a common one in which the
amount of neurons is the same in every layer, and a bottleneck-shaped architecture in which the middle
layer has fewer neurons. Grézl et al. claim [Grézl07] that Bottleneck architecture is more suitable to
learn high-level features than common one, and that it reduces overfitting. Moreover, following Zhou and
Lerch results [Zhou15], it leads to better results for chord recognition than a common architecture.'''

A1-incomplete dominant-seventh chord


In [774]:
def to_chords(inChords, inFrmt):
    
    if inFrmt == 'madChordDF':
        outChords = inChords.iloc[:,0:2]
        notClass = [float('nan'), None, float('nan'), None, None, None, None]
        dels = []
        pitchClasses = range(NUM_PITCH_CLASSES) # Integer vales 0-11, where C=0, C#=1, D=2...B=11 per M21
        newLines = []
        
        for i in range(inChords.shape[0]):
            newLine = []
            
            # pitch class
            if inChords.iloc[i,2][0] == 'N':
                newLines.append(notClass)
                dels.append(i)
                continue
                '''for lines identified as 'N', add placeholder which will be subsequently deleted. these are 
                chords for which the onset is known, but the classification is not known (trained). by deleting
                we re losing info. however, evaluation chokes on records w/ nans or nones. further, given how 
                effective note rnn is at spotting chord onset AND classifying chords, chord CNN/DNN will be 
                used to augment its predictions'''
            elif inChords.iloc[i,2][1] == ':': 
                newLine.append(pitchClasses[PITCH_CLASSES.index(inChords.iloc[i,2][0])])
                # accidental
                newLine.append('accidental natural')
            elif inChords.iloc[i,2][1] == '#':
                '''flats "-" madmon trained their network w/ 25 classes representing:
                1. 12 pitch classes: 'A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#'
                2. 2 qualities 'maj', 'min'
                3. N for none of the above
                
                Problem is that, while they occur often in music, flats aren't included. Looking at akpnbcht,
                see that E- and B- chords are classed as D# and A# by the algorithm. Specifically:
                1. of 13.5k chords, 1.9k (14%) were flats. 1,112 E-, 820 B-. 
                2. of 1,112 E-, 208 (100% of those classed by the algo) were classed as D#. 
                3. of 820 B-, 128 (100% of those classed by the algo) were classed as A#. 
                4. of mad chords classed D, 376 were classed by m21 as D, 208 as E.
                5. of mad chords classed A, 354 were classed by m21 as A, 128 as B.
                
                so, assuming akpnbcht is a representative dataset:
                1. mad classifies flats as next pitch class down
                2. this requires changes to:
                    a. the mad chord outputs to bring them in inline w/ m21-based classifications (below)
                    b. the m21-based y records
                '''
                if inChords.iloc[i,2][0] == 'D':
                    newLine.append(pitchClasses[PITCH_CLASSES.index('E')])
                    newLine.append('accidental flat')
                elif inChords.iloc[i,2][0] == 'A':
                    newLine.append(pitchClasses[PITCH_CLASSES.index('B')])
                    newLine.append('accidental flat')
                else: 
                    newLine.append(pitchClasses[PITCH_CLASSES.index(inChords.iloc[i,2][0:1])])
                    newLine.append('accidental sharp')
            elif inChords.iloc[i,2][1] == '-':
                newLine.append(pitchClasses[PITCH_CLASSES.index(inChords.iloc[i,2][0])+1])
                newLine.append('accidental flat')
            else:
                print('unrecognized accidental on chord input file at line', i); end
            
            # octave
            newLine.append(-10)
            
            # triad
            newLine.append(True)
            
            # quality
            s = inChords.iloc[i,2] 
            try: 
                start = s.find(':') + 1
                end = len(s)
                found = s[start:end]
            except:
                print("chord record without maj/min quality:", i)
                end
            if found == 'maj': newLine.append("major")
            elif found == 'min': newLine.append("minor")
            else: print("chord record with non maj/min quality:", i); end
            
            # m21 pitched common name
            newLine.append(None)
            
            # mad chord label
            newLine.append(inChords.iloc[i,2])
            newLines.append(newLine)
    
    elif inFrmt == "m21noteDF":
        # going from stream of notes to m21 chords
        outChords = pd.DataFrame(inChords.drop_duplicates(subset="OnsetTime", keep='first'))
        outChords = outChords.iloc[:,0:2]
        dels = []; newLines = []
        
        for i in range(outChords.shape[0]):
            newLine = []
            curPitches = inChords.MidiPitch[inChords.OnsetTime == outChords.OnsetTime.iloc[i]]
            crd = m21.chord.Chord(np.array(curPitches))
            root = crd.root()
            newLine.append(root.pitchClass)
                
            s = str(root.accidental)
            
            try: 
                start = s.find('<') + 1
                end = s.find('>', start)
                found = s[start:end]
            except:
                found = root.accidental
                print("accidental w/ out < or >:", i)
                print(root.accidental)
    
            newLine.append(found)
            newLine.append(root.octave)
            newLine.append(crd.containsTriad())
            newLine.append(crd.quality)
            newLine.append(crd.pitchedCommonName)
                
            if crd.containsTriad() == True:
                '''based on the assumption that the basic unit of chord construction is the major/minor triad, 
                I check for triad here, then determine the appropriate (as near as I can tell) madmom root. This
                reverses the process used to map from madmom chords to m21 chords above. 
                '''
                # if root is flat,...
                if root.accidental == 'accidental flat':
                    # ... move down one note
                    if root.pitchClass >= 1:
                        pcStr = PITCH_CLASSES[root.pitchClass-1]
                    else:
                        pcStr = PITCH_CLASSES[-1]
                else:
                    pcStr = PITCH_CLASSES[root.pitchClass]
                    
                if crd.quality == "major": madChordLabel = pcStr + ':maj'
                elif crd.quality == "minor": madChordLabel = pcStr + ':min'
                else: madChordLabel = 'N'
                        
            else:
                madChordLabel = 'N'
                    
            newLine.append(madChordLabel)
            newLines.append(newLine)
            
    newLines = pd.DataFrame(newLines, index=outChords.index,
                            columns=["m21RootPitchClass", "m21RootPitchAccidental", "m21RootOctave",
                                     "m21ContainsTriad", "m21ChordQuality", "m21PitchedCommonName",
                                     "madChordLabel"])
    
    outChords = outChords.join(newLines)
    outChords = outChords.drop(outChords.index[dels])
        
    return(outChords)
    
# test it
#dnn_chord_rec(inWav='data/wip/tmp/MAPS_MUS-bach_846_AkPnBcht.wav', 
#              savTo='data/wip/tmp/MAPS_MUS-bach_846_AkPnBcht.chords.dnn.txt')

In [623]:
# for clearning up chords

tmp = glob.glob('data/wip/AkPnBcht/MUS/*.chord.y.txt')
# 'data/wip/AkPnBcht/MUS/*.chords.dnn.txt'
rcdCtr = 0
flatCtr = 0
m21Ltr = []
madLtr = []
m21dSharpLtr = []
m21aSharpLtr = []

for line in tmp:
    madChords = load_chords(line)
    
    for i in range(madChords.shape[0]):
        rcdCtr = rcdCtr + 1
        if madChords.iloc[i,7][1] == '-':
            flatCtr = flatCtr +1
            m21Ltr.append(madChords.iloc[i,7][0]) 
            madLtr.append(madChords.iloc[i,8][0])

        if madChords.iloc[i,8][0] == 'D':
            m21dSharpLtr.append(madChords.iloc[i,7][0:1]) 
            
        if madChords.iloc[i,8][0] == 'A':
            m21aSharpLtr.append(madChords.iloc[i,7][0:1]) 
            
    
print("rcds:", rcdCtr, "flats:", flatCtr, "pct:", flatCtr / rcdCtr)
print

print("when classed '-' by m21, what is the madmom class?")
for i in range(len(PITCH_CLASSES)):
    print(PITCH_CLASSES[i], ": m21:", m21Ltr.count(PITCH_CLASSES[i]), ": mad:", madLtr.count(PITCH_CLASSES[i]))
print
    
print("when classed D# by madmom, what is the m21 class?")
for i in range(len(PITCH_CLASSES)):
    print(PITCH_CLASSES[i], ":" "m21:", m21dSharpLtr.count(PITCH_CLASSES[i]))
print

print("when classed A# by madmom, what is the m21 class?")
for i in range(len(PITCH_CLASSES)):
    print(PITCH_CLASSES[i], ":" "m21:", m21aSharpLtr.count(PITCH_CLASSES[i]))
        

('rcds:', 13530, 'flats:', 1932, 'pct:', 0)

when classed '-' by m21, what is the madmom class?
('C', ': m21:', 0, ': mad:', 0)
('C#', ': m21:', 0, ': mad:', 0)
('D', ': m21:', 0, ': mad:', 208)
('D#', ': m21:', 0, ': mad:', 0)
('E', ': m21:', 1112, ': mad:', 0)
('F', ': m21:', 0, ': mad:', 0)
('F#', ': m21:', 0, ': mad:', 0)
('G', ': m21:', 0, ': mad:', 0)
('G#', ': m21:', 0, ': mad:', 0)
('A', ': m21:', 0, ': mad:', 128)
('A#', ': m21:', 0, ': mad:', 0)
('B', ': m21:', 820, ': mad:', 0)

when classed D# by madmom, what is the m21 class?
('C', ':m21:', 0)
('C#', ':m21:', 0)
('D', ':m21:', 376)
('D#', ':m21:', 0)
('E', ':m21:', 208)
('F', ':m21:', 0)
('F#', ':m21:', 0)
('G', ':m21:', 0)
('G#', ':m21:', 0)
('A', ':m21:', 0)
('A#', ':m21:', 0)
('B', ':m21:', 0)
when classed A# by madmom, what is the m21 class?
('C', ':m21:', 0)
('C#', ':m21:', 0)
('D', ':m21:', 0)
('D#', ':m21:', 0)
('E', ':m21:', 0)
('F', ':m21:', 0)
('F#', ':m21:', 0)
('G', ':m21:', 0)
('G#', ':m21:', 0)
('A', ':m21:',

In [642]:
tmp = glob.glob('data/wip/AkPnBcht/MUS/*.chords.cnn.txt')
pitchClasses = range(12)

for line in tmp:
    
    inChords = load_chords(line)
    
    for i in range(inChords.shape[0]):
        
        if inChords.iloc[i,8][1] == '#':
            if inChords.iloc[i,8][0] == 'D':
                print("in D, assigning:", pitchClasses.index[PITCH_CLASSES == 'E'], "s/b 5")
                inChords.iloc[i,2] = pitchClasses[PITCH_CLASSES == 'E']
                inChords.iloc[i,3] = 'accidental flat'
            elif inChords.iloc[i,8][0] == 'A':
                print("in A, assigning:", pitchClasses.index[PITCH_CLASSES == 'B'], "s/b 11")
                inChords.iloc[i,2] = pitchClasses[PITCH_CLASSES == 'B']
                inChords.iloc[i,3] = 'accidental flat'
                
    save_chords(line, inChords)

('in D, assigning:', 0, 's/b 5')
('in A, assigning:', 0, 's/b 11')
('in A, assigning:', 0, 's/b 11')
('in D, assigning:', 0, 's/b 5')
('in A, assigning:', 0, 's/b 11')
('in D, assigning:', 0, 's/b 5')
('in D, assigning:', 0, 's/b 5')
('in D, assigning:', 0, 's/b 5')
('in A, assigning:', 0, 's/b 11')
('in D, assigning:', 0, 's/b 5')
('in D, assigning:', 0, 's/b 5')
('in A, assigning:', 0, 's/b 11')
('in A, assigning:', 0, 's/b 11')
('in D, assigning:', 0, 's/b 5')
('in A, assigning:', 0, 's/b 11')
('in D, assigning:', 0, 's/b 5')
('in A, assigning:', 0, 's/b 11')
('in A, assigning:', 0, 's/b 11')
('in D, assigning:', 0, 's/b 5')
('in A, assigning:', 0, 's/b 11')
('in D, assigning:', 0, 's/b 5')
('in D, assigning:', 0, 's/b 5')
('in D, assigning:', 0, 's/b 5')
('in A, assigning:', 0, 's/b 11')
('in D, assigning:', 0, 's/b 5')
('in A, assigning:', 0, 's/b 11')
('in D, assigning:', 0, 's/b 5')
('in A, assigning:', 0, 's/b 11')
('in A, assigning:', 0, 's/b 11')
('in A, assigning:', 0, 's/b 

In [584]:

def txt_to_y(inFile, mode="savedFile", out_fmt='runTime', wav_dur=0, wav_frames=0):
    '''TODO: implement chord recognition window s/t notes w/in window are considered components of chord.
    only do this if you see material chord misclassifications in normal application use.'''
    # valid y_modes: "pitch_only", "chord_only", "music"
    
    '''translates text files w/ onset, offset and pitch into two Y matricies:
    #  1. y_notes: [n, 89] vect of binaries representing:
    #     a. notes C3 on piano keyboard A1->C#9 (midi 21-109)
    #     b. "not target class" including notes/pitches outside range AND in-range chords formed by 2+ keys
    #  2. y_chords: [n, 120] vect of the "major" and "minor" triads (some of the most common chords in western music):
    #     a. 12 pitch classes i.e., C, C#, D, D#, E, F, F#, G, G#, A, A#, B
    #     b. 5 (of total 7.5) octaves i.e., C in 4th octave: C4
    #     c. 1 (of total 6 in db) key combinations i.e., triad (not 2, 4, 5, 6 or 7 key combinations)
    #     d. major (root pitch, +4 pitches, +3 pitches) or  minor (root pitch, +3, +4) quality 
    #        i.e., C major triad 4th octave: C4, E4, G4. C minor triad 4th octave: C4, D#4, G4
    #     e. NOT IMPLEMENTED: 2 inversions that are defined by the lowest note in the chord
    #        i.e., C4, E4, G4 becomes E4, G4, C5 in the first inversion  
    #     f. "not target class" including single notes and the multitude of less common 2+ note chords
    #     other options: http://www.daigleharp.com/Images/Help%20Files/commonchordsforautoharp.pdf, 
    http://www.hooktheory.com/blog/i-analyzed-the-chords-of-1300-popular-songs-for-patterns-this-is-what-i-found/'''
    
    if mode == "savedFile":
        # reads text file into dataframe, sort and round
        lines = read_maps_note_file(inFile)
    else:
        lines = inFile
    
    chord_rcds_fl = 0; in_chord_fl = 0; pitch_rcds_fl = 0; rval = 0
    
    # if single line file, save pitch
    if lines.shape[0] == 1:
        active_pitches = lines.iloc[[0]]
        pitch_rcds_fl = 1
    
    # otherwise, step thru lines assigning notes to pitches or chords
    else:
        for i in range(1,lines.shape[0]):
            
            # process pitches: madmom RNN good at extracting notes even from chords. 
            # so, process all rcds as note records
            if pitch_rcds_fl == 0:
                # ...then instantiate note array using prior line
                active_pitches = lines.iloc[[i-1]]
                pitch_rcds_fl = 1
                    
            # ... and note array already started, then append line to note array
            else:
                active_pitches = active_pitches.append(lines.iloc[[i-1]])
                
            # if i is the last line in input array, move it to pitch array also
            if i == (lines.shape[0]-1):
                if pitch_rcds_fl == 0:
                    active_pitches = lines.iloc[[i]]
                else:
                    active_pitches = active_pitches.append(lines.iloc[[i]])
                        
            # process chords: if note record has same onset (w/in rounding tolerance of 0.01) as prior...
            if lines.iloc[i,0] == lines.iloc[i-1,0]: #and lines.iloc[i,1] == lines.iloc[i-1,1]:
                # ... and it's the first chord in piece,...
                if chord_rcds_fl == 0:
                    # ...then instantiate chord array using the prior pitch (line)
                    active_chords = lines.iloc[[i-1]]
                    chord_rcds_fl = 1; in_chord_fl = 1
                
                # otherwise, append the prior pitch (line) to chord array
                else:
                    active_chords = active_chords.append(lines.iloc[[i-1]])
                    in_chord_fl = 1
                
                # if last line in input array (and it's same as prior), move it to chord array
                if i == (lines.shape[0]-1):
                    active_chords = active_chords.append(lines.iloc[[i]])
                    
            # so, current line doesn't have same onset...
            else:
                #...but you were in a chord... 
                if in_chord_fl == 1:
                    #...append prior pitch (line) to the chord array.
                    active_chords = active_chords.append(lines.iloc[[i-1]])
                    in_chord_fl = 0
                
                    # if last line in input array (and you're in a chord), move it to chord array
                    if i == (lines.shape[0]-1):
                        active_chords = active_chords.append(lines.iloc[[i]])
    
    if out_fmt == "runTime":
        if(pitch_rcds_fl == 0):
            active_pitches = None
            
        if(chord_rcds_fl == 0):
            active_chords = None
        
        else:
            chordsDF = to_chords(active_chords, inFrmt = "m21noteDF")
            #print("4c. after", i, "iterations:", time.time())
        
        return(active_pitches, chordsDF)
    
    elif out_fmt == "oneHot":
        
        # format time index w/ slices = wave frame sample rate        
        time_ctr = 0
        time_incr = float(wav_dur) / wav_frames
        time_idx = []
    
        for k in range(wav_frames): 
            time_idx.append(np.round(time_ctr,2))
            time_ctr = time_ctr + time_incr
    
        # initialize y matrices    
        Y_pitch = pd.DataFrame(np.zeros((len(time_idx), NUM_PITCH_CLASSES), dtype=int),
                               index = time_idx, columns = PITCH_CLASSES)
        Y_pitch.iloc[:,0] = 1 # set NOT_TGT_CLASS on as default
        
        Y_chord = pd.DataFrame(np.zeros((len(time_idx), NUM_CHORD_CLASSES), dtype=int),
                               index = time_idx, columns = CHORD_CLASSES)
        Y_chord.iloc[:,0] = 1 # set NOT_TGT_CLASS on as default
    
        time_idx = pd.DataFrame(time_idx)

        # step thru active, single pitch records
        if(pitch_rcds_fl > 0):
        
            for i in range(active_pitches.shape[0]):
               
                # ...find the ids of all time indexes that fall after the onset...
                more = time_idx[time_idx[0] >= active_pitches.iloc[i,0]].index.tolist()
        
                #...and the ids of all time indexes that fall before the offset
                less = time_idx[time_idx[0] < active_pitches.iloc[i,1]].index.tolist()
        
                # the intersection are the id's of time indexes where a pitch was active
                net = np.intersect1d(more, less, assume_unique=False)
        
                # flip the class variable for each time index
                for j in range(len(net)):
                    # if it's a valid pitch...
                    if active_pitches.iloc[i,2] in PITCH_CLASSES:
                        # ...flip the corresponding pitch column
                        Y_pitch.loc[time_idx.iloc[net[j],0], int(active_pitches.iloc[i,2])] = 1
                        Y_pitch.loc[time_idx.iloc[net[j],0], NOT_TGT_LABEL] = 0
            
        # step thru active chord records
        if(chord_rcds_fl > 0):
            uniq_onset = active_chords.OnsetTime.unique()
          
            for i in range(uniq_onset.shape[0]):
                cur_pitches = active_chords[active_chords.iloc[:,0] == uniq_onset[i]]
                cur_chord = m21.chord.Chord(np.array(cur_pitches.iloc[:,2])).pitchedCommonName
            
                # ...find the ids of all time indexes that fall after the onset...
                more = time_idx[time_idx[0] >= uniq_onset[i]].index.tolist()
                            
                #...and the ids of all time indexes that fall before the offset
                less = time_idx[time_idx[0] < cur_pitches.iloc[0,1]].index.tolist()
            
                # the intersection are the id's of time indexes where a pitch was active
                net = np.intersect1d(more, less, assume_unique=False)
        
                # if it's a valid chord,...
                if cur_chord in CHORD_CLASSES:
                    # ...cycle thru time indexes...
                    for j in range(len(net)):
                        # ...flipping the corresponding chord column
                        Y_chord.loc[time_idx.iloc[net[j],0], cur_chord] = 1
                        Y_chord.loc[time_idx.iloc[net[j],0], NOT_TGT_LABEL] = 0
                    
                    rval = 1
                
        #return([rval])
        return(Y_pitch, Y_chord)
        
# test music
#ptch, crd = txt_to_y('data/maps/AkPnBcht/MUS/MAPS_MUS-bach_846_AkPnBcht.txt')

In [47]:
def read_maps_note_file(txt_file):
    # reads text file into dataframe, sort and round
    
    lines = [line.rstrip('\n').split('\t') for line in open(txt_file, 'U')]
    
    headers = lines[0]; lines = lines[1:len(lines)]
    
    lines = pd.DataFrame(lines, columns=[headers[0], headers[1], 
                                         headers[2]]).convert_objects(convert_numeric=True)
    
    lines = lines.round({'OnsetTime': 2, 'OffsetTime': 2, 'MidiPitch': 0})
    
    lines = lines.sort_values(['OnsetTime', 'MidiPitch'], # 'OffsetTime', 
                              axis=0, ascending=True, inplace=False, 
                              kind='quicksort', na_position='last')
    
    '''sort is tricky. some chord notes have same onset, different offsets. technically, the shorter note
    should probably appear first (i.e., ascending sort: onset, offset, pitch). problem is you get an 
    expanding set of chord classes most of which don't sound musically different (i.e., imperceptible 
    differences in offset). so, i'm sorting above using ascending: onset, midi (after pitch, offset does not matter)''' 
    
    
    return(lines)

## Load streams

In [ ]:
# from microphone to 

## Evaluate

### Notes

In [770]:
# NOTES
# 'data/wip/AkPnBcht/MUS/*.note.y.txt'
# 'data/wip/AkPnBcht/MUS/*.notes.rnn.txt'
noteAnnots = glob.glob('data/wip/AkPnBcht/UCHO/I60-68/*.note.y.txt')
noteDetects = glob.glob('data/wip/AkPnBcht/UCHO/I60-68/*.notes.rnn.txt')

if len(noteAnnots) == len(noteDetects):
    eval_objs = []
    for i in range(len(noteAnnots)):
        annotations = mad.features.notes.load_notes(noteAnnots[i])
        detections = mad.features.notes.load_notes(noteDetects[i])
        
        eval_objs.append(mad.evaluation.notes.NoteEvaluation(detections, annotations, window=0.025, delay=0))
    
    print('***** Note performance:')
    print(mad.evaluation.notes.NoteSumEvaluation(eval_objs, name=None).tostring())
    
else:
    print("missing Annots or Detects")
    end


missing Annots or Detects


### Chords

In [725]:
# evaluate results

# onset evaluation function
def onset_evaluation(detections, annotations, window=WINDOW):
    """
    Determine the true/false positive/negative detections.
    Parameters
    ----------
    detections : numpy array
        Detected notes.
    annotations : numpy array
        Annotated ground truth notes.
    window : float, optional
        Evaluation window [seconds].
    Returns
    -------
    tp : numpy array, shape (num_tp,)
        True positive detections.
    fp : numpy array, shape (num_fp,)
        False positive detections.
    tn : numpy array, shape (0,)
        True negative detections (empty, see notes).
    fn : numpy array, shape (num_fn,)
        False negative detections.
    errors : numpy array, shape (num_tp,)
        Errors of the true positive detections wrt. the annotations.
    Notes
    -----
    The returned true negative array is empty, because we are not interested
    in this class, since it is magnitudes bigger than true positives array.
    """
    # make sure the arrays have the correct types and dimensions
    detections = np.asarray(detections) # , dtype=np.float
    annotations = np.asarray(annotations) # , dtype=np.float
    # TODO: right now, it only works with 1D arrays
    if detections.ndim > 1 or annotations.ndim > 1:
        raise NotImplementedError('please implement multi-dim support')

    # init TP, FP, FN and errors
    tp = np.zeros(0)
    fp = np.zeros(0)
    tn = np.zeros(0)  # we will not alter this array
    fn = np.zeros(0)
    errors = np.zeros(0)

    # if neither detections nor annotations are given
    if len(detections) == 0 and len(annotations) == 0:
        # return the arrays as is
        return tp, fp, tn, fn, errors
    # if only detections are given
    elif len(annotations) == 0:
        # all detections are FP
        return tp, detections, tn, fn, errors
    # if only annotations are given
    elif len(detections) == 0:
        # all annotations are FN
        return tp, fp, tn, annotations, errors

    # window must be greater than 0
    if float(window) <= 0:
        raise ValueError('window must be greater than 0')

    # sort the detections and annotations
    det = np.sort(detections)
    ann = np.sort(annotations)
    # cache variables
    det_length = len(detections)
    ann_length = len(annotations)
    det_index = 0
    ann_index = 0
    # iterate over all detections and annotations
    while det_index < det_length and ann_index < ann_length:
        # fetch the first detection
        d = det[det_index]
        # fetch the first annotation
        a = ann[ann_index]
        # compare them
        if abs(d - a) <= window:
            # TP detection
            tp = np.append(tp, d)
            # append the error to the array
            errors = np.append(errors, d - a)
            # increase the detection and annotation index
            det_index += 1
            ann_index += 1
        elif d < a:
            # FP detection
            fp = np.append(fp, d)
            # increase the detection index
            det_index += 1
            # do not increase the annotation index
        elif d > a:
            # we missed a annotation: FN
            fn = np.append(fn, a)
            # do not increase the detection index
            # increase the annotation index
            ann_index += 1
        else:
            # can't match detected with annotated onset
            raise AssertionError('can not match % with %', d, a)
    # the remaining detections are FP
    fp = np.append(fp, det[det_index:])
    # the remaining annotations are FN
    fn = np.append(fn, ann[ann_index:])
    # check calculations
    if len(tp) + len(fp) != len(detections):
        raise AssertionError('bad TP / FP calculation')
    if len(tp) + len(fn) != len(annotations):
        raise AssertionError('bad FN calculation')
    if len(tp) != len(errors):
        raise AssertionError('bad errors calculation')
    # convert to numpy arrays and return them
    return np.array(tp), np.array(fp), tn, np.array(fn), np.array(errors)


class MultiClassEvaluation(mad.evaluation.Evaluation):
    """
    Evaluation class for measuring Precision, Recall and F-measure based on
    2D numpy arrays with true/false positive/negative detections.
    Parameters
    ----------
    tp : list of tuples or numpy array, shape (num_tp, 2)
        True positive detections.
    fp : list of tuples or numpy array, shape (num_fp, 2)
        False positive detections.
    tn : list of tuples or numpy array, shape (num_tn, 2)
        True negative detections.
    fn : list of tuples or numpy array, shape (num_fn, 2)
        False negative detections.
    name : str
        Name to be displayed.
    Notes
    -----
    The second item of the tuples or the second column of the arrays denote
    the class the detection belongs to.
    """
    def __init__(self, tp=None, fp=None, tn=None, fn=None, **kwargs):
        # set default values
        if tp is None:
            tp = np.zeros((0, 2))
        if fp is None:
            fp = np.zeros((0, 2))
        if tn is None:
            tn = np.zeros((0, 2))
        if fn is None:
            fn = np.zeros((0, 2))
        super(MultiClassEvaluation, self).__init__(**kwargs)
        self.tp = np.asarray(tp) # , dtype=np.float
        self.fp = np.asarray(fp) # , dtype=np.float
        self.tn = np.asarray(tn) # , dtype=np.float
        self.fn = np.asarray(fn) # , dtype=np.float

    def tostring(self, verbose=False, **kwargs):
        """
        Format the evaluation metrics as a human readable string.
        Parameters
        ----------
        verbose : bool
            Add evaluation for individual classes.
        Returns
        -------
        str
            Evaluation metrics formatted as a human readable string.
        """
        ret = ''

        if verbose:
            # extract all classes
            classes = []
            if self.tp.any():
                classes = np.append(classes, np.unique(self.tp[:, 1]))
            if self.fp.any():
                classes = np.append(classes, np.unique(self.fp[:, 1]))
            if self.tn.any():
                classes = np.append(classes, np.unique(self.tn[:, 1]))
            if self.fn.any():
                classes = np.append(classes, np.unique(self.fn[:, 1]))
            for cls in sorted(np.unique(classes)):
                # extract the TP, FP, TN and FN of this class
                tp = self.tp[self.tp[:, 1] == cls]
                fp = self.fp[self.fp[:, 1] == cls]
                tn = self.tn[self.tn[:, 1] == cls]
                fn = self.fn[self.fn[:, 1] == cls]
                # evaluate them
                e = Evaluation(tp, fp, tn, fn, name='Class %s' % cls)
                # append to the output string
                ret += '  %s\n' % e.tostring(verbose=False)
        # normal formatting
        ret += 'Annotations: %5d TP: %5d FP: %4d FN: %4d ' \
               'Precision: %.3f Recall: %.3f F-measure: %.3f Acc: %.3f' % \
               (self.num_annotations, self.num_tp, self.num_fp, self.num_fn,
                self.precision, self.recall, self.fmeasure, self.accuracy)
        # return
        return ret
    
    
def ChordOnsetEvaluation(detections, annotations, window):
    
    # init TP, FP, TN and FN lists
    tp = np.zeros((0, 2))
    fp = np.zeros((0, 2))
    tn = np.zeros((0, 2))  # this will not be altered
    fn = np.zeros((0, 2))
    errors = np.zeros((0, 2))
    
    # get a list of all chords detected / annotated
    chords = np.unique(np.concatenate((detections[:,1],
                                       annotations[:,1]))).tolist()
    #print("chords:")
    #print(chords)
    # iterate over all chords
    for chord in chords:
        # perform normal onset detection on each chord
        det = detections[detections[:, 1] == chord]
        ann = annotations[annotations[:, 1] == chord]
        tp_, fp_, _, fn_, err_ = onset_evaluation(det[:, 0], ann[:, 0], window)
        
        # convert returned arrays to lists and append the detections and
        # annotations to the correct lists
        tp = np.vstack((tp, det[np.in1d(det[:, 0], tp_)]))
        fp = np.vstack((fp, det[np.in1d(det[:, 0], fp_)]))
        fn = np.vstack((fn, ann[np.in1d(ann[:, 0], fn_)]))
        
        # append the chord number to the errors
        err_ = np.vstack((np.array(err_),
                          np.repeat(np.asarray([chord]), len(err_)))).T
        errors = np.vstack((errors, err_))
        
    # check calculations
    #print("len(tp):", len(tp))
    #print("len(fp):", len(fp))
    #print("len(detections:)", len(detections))
    if len(tp) + len(fp) != len(detections):
        raise AssertionError('bad TP / FP calculation')
    if len(tp) + len(fn) != len(annotations):
        raise AssertionError('bad FN calculation')
    if len(tp) != len(errors):
        raise AssertionError('bad errors calculation')
        
    # sort the arrays
    # Note: The errors must have the same sorting order as the TPs, so they
    #       must be done first (before the TPs get sorted)
    errors = errors[tp[:, 0].argsort()]
    tp = tp[tp[:, 0].argsort()]
    fp = fp[fp[:, 0].argsort()]
    fn = fn[fn[:, 0].argsort()]
    
    # return the arrays
    return tp, fp, tn, fn, errors
        
        
# for chord evaluation with Precision, Recall, F-measure use the Evaluation
# class and just define the evaluation function
# TODO: extend to also report the measures without octave errors
class ChordEvaluation(MultiClassEvaluation):
    """
        Evaluation class for measuring Precision, Recall and F-measure of chords.
        
        Parameters
        ----------
        detections : str, list or numpy array
            Detected chords.
            
        annotations : str, list or numpy array
            Annotated ground truth chords.
            
        window : float, optional
            F-measure evaluation window [seconds]
            
        delay : float, optional
            Delay the detections `delay` seconds for evaluation.
    """
    
    def __init__(self, detections, annotations, evalCols, window=WINDOW, delay=0,
                 **kwargs):
        
        # load the chord detections and annotations
        detections = pd.read_table(detections)
        annotations = pd.read_table(annotations)
        
        # shift the detections if needed
        if delay != 0:
            detections.iloc[:, 0] += delay
        
        # build input array        
        if len(evalCols) == 2:
            outDet = np.array(detections.iloc[:,evalCols])
            outAnn = np.array(annotations.iloc[:,evalCols])
            
        else:
            outDet = detections.iloc[:,evalCols[0]]
            outAnn = annotations.iloc[:,evalCols[0]]
            
            classDet = detections.iloc[:,evalCols[1]].tolist()
            classAnn = annotations.iloc[:,evalCols[1]].tolist()

            for i in range(2,len(evalCols)):
                newDet = detections.iloc[:,evalCols[i]]
                newAnn = annotations.iloc[:,evalCols[i]]
                
                for j in range(len(classDet)):
                    classDet[j] = str(classDet[j]) + str(newDet[j])
                
                for j in range(len(classAnn)):
                    classAnn[j] = str(classAnn[j]) + str(newAnn[j])
            
            outDet = np.array(pd.DataFrame({'a': outDet, 'b':classDet}))
            outAnn = np.array(pd.DataFrame({'a': outAnn, 'b':classAnn}))
        
        # evaluate onsets (passing only onset and classifier)
        perf = ChordOnsetEvaluation(outDet, outAnn, window)
        tp, fp, tn, fn, errors = perf

        super(ChordEvaluation, self).__init__(tp, fp, tn, fn, **kwargs)
        self.errors = errors
        
        # save them for the individual chord evaluation
        self.detections = detections
        self.annotations = annotations
        self.window = window
    
    @property
    def mean_error(self):
        """Mean of the errors."""
        warnings.warn('mean_error is given for all chords, this will change!')
        if len(self.errors) == 0:
            return np.nan
        else:
            return np.nanmean(self.errors[:,0].astype(np.float))
    
    @property
    def std_error(self):
        """Standard deviation of the errors."""
        warnings.warn('std_error is given for all chords, this will change!')
        if len(self.errors) == 0:
            return np.nan
        else:
            return np.nanstd(self.errors[:,0].astype(np.float))
    
    def tostring(self, chords=False, **kwargs):
        """
            Parameters
            ----------
            chords : bool, optional
                Display detailed output for all individual chords.
                
            Returns
            -------
            str
                Evaluation metrics formatted as a human readable string.
        """
        
        ret = ''
        if self.name is not None:
            ret += '%s\n  ' % self.name
        
        # add statistics for the individual chord
        if chords:
            
            # determine which chords are present
            chords = []
            if self.tp.any():
                chords = np.append(chords, np.unique(self.tp[:, 1]))
            if self.fp.any():
                chords = np.append(chords, np.unique(self.fp[:, 1]))
            if self.tn.any():
                chords = np.append(chords, np.unique(self.tn[:, 1]))
            if self.fn.any():
                chords = np.append(chords, np.unique(self.fn[:, 1]))

            # evaluate them individually
            for chord in sorted(np.unique(chords)):
                
                # detections and annotations for this chord (only onset times)
                det = self.detections[self.detections[:, 1] == chord][:, 0]
                ann = self.annotations[self.annotations[:, 1] == chord][:, 0]
                name = 'chord %s' % chord
                e = mad.evaluation.onsets.OnsetEvaluation(det, ann, self.window, name=name)
                
                # append to the output string
                ret += '  %s\n' % e.tostring(chords=False)
                    
        # normal formatting
        ret += 'chords: %5d TP: %5d FP: %4d FN: %4d ' \
            'Precision: %.3f Recall: %.3f F-measure: %.3f ' \
                'Acc: %.3f mean: %5.1f ms std: %5.1f ms' % \
                    (self.num_annotations, self.num_tp, self.num_fp, self.num_fn,
                     self.precision, self.recall, self.fmeasure, self.accuracy,
                     self.mean_error * 1000., self.std_error * 1000.)
        # return
        return ret


class ChordSumEvaluation(mad.evaluation.SumEvaluation, ChordEvaluation):
    """
        Class for summing chord evaluations.
    """
    
    @property
    def errors(self):
        """Errors of the true positive detections wrt. the ground truth."""
        if not self.eval_objects:
            # return empty array
            return np.zeros((0, 2))
        return np.concatenate([e.errors for e in self.eval_objects])


class ChordMeanEvaluation(mad.evaluation.MeanEvaluation, ChordSumEvaluation):
    """
        Class for averaging chord evaluations.
    """
    
    @property
    def mean_error(self):
        """Mean of the errors."""
        warnings.warn('mean_error is given for all chords, this will change!')
        return np.nanmean([e.mean_error for e in self.eval_objects])
    
    @property
    def std_error(self):
        """Standard deviation of the errors."""
        warnings.warn('std_error is given for all chords, this will change!')
        return np.nanmean([e.std_error for e in self.eval_objects])
    
    def tostring(self, **kwargs):
        """
            Format the evaluation metrics as a human readable string.
            
            Returns
            -------
            str
                Evaluation metrics formatted as a human readable string.
        """
        
        # format with floats instead of integers
        ret = ''
        if self.name is not None:
            ret += '%s\n  ' % self.name
        ret += 'Chords: %5.2f TP: %5.2f FP: %5.2f FN: %5.2f ' \
            'Precision: %.3f Recall: %.3f F-measure: %.3f ' \
                'Acc: %.3f mean: %5.1f ms std: %5.1f ms' % \
                    (self.num_annotations, self.num_tp, self.num_fp, self.num_fn,
                     self.precision, self.recall, self.fmeasure, self.accuracy,
                     self.mean_error * 1000., self.std_error * 1000.)
        return ret



In [763]:
# CHORDS

# Evaluate chords
'''In music theory, the concept of root denotes the idea that a chord can be represented and named by one of its notes. It is linked to harmonic thinking, that is, to the idea that vertical aggregates of notes can form a single unit, a chord. It is in this sense that one can speak of a "C chord", or a "chord on C", a chord built from "C" and of which the note (or pitch) "C" is the root. When a C chord is referred to in Classical music or popular music without a reference to what type of chord it is (either Major or minor, in most cases), this chord is assumed to be a C major triad, which contains the notes C, E and G. The root needs not be the bass note, the lowest note of the chord: the concept of root is linked to that of the inversion of chords, which is derived from the notion of invertible counterpoint. In this concept, chords can be inverted while still retaining their root.

'''

chdRnnDet = glob.glob('data/wip/AkPnBcht/MUS/*.chords.rnn.txt')
chdCnnDet = glob.glob('data/wip/AkPnBcht/MUS/*.chords.cnn.txt')
chdDnnDet = glob.glob('data/wip/AkPnBcht/MUS/*.chords.dnn.txt')
chdAnnot = glob.glob('data/wip/AkPnBcht/MUS/*.chords.y.txt')

fSets = [chdRnnDet, chdCnnDet, chdDnnDet]
fSetTtl = ['Note RNN Chord Pred', 'CNN Chord Preds', 'DNN Chord Preds']
evalCols = [(0,2),(0,2,3),(0,4),(0,2,3,4),(0,6),(0,8),(0,7)]

for h in range(len(fSets)):
    pcObjs = []; pObjs = []; oObjs = []; nObjs = []; qObjs = []; madObjs = []; m21Objs = []
    
    if h == 0: 
        delay = 0.01 # RNN delay
        window = 0.05
    elif h == 1:
        delay = 0.1 # CNN delay
        window = 0.5
    elif h == 2:
        delay = 0.01 # DNN delay
        window = 0.5
    
    for i in range(len(fSets[h])): 
        # pitch class performance
        pcObjs.append(ChordEvaluation(fSets[h][i], chdAnnot[i], evalCols[0], window=window, delay=delay))
        
        # pitch (class + accidental) performance
        pObjs.append(ChordEvaluation(fSets[h][i], chdAnnot[i], evalCols[1], window=window, delay=delay))
        
        # octave performance
        oObjs.append(ChordEvaluation(fSets[h][i], chdAnnot[i], evalCols[2], window=window, delay=delay))
        
        # note (pitch + octave) performance
        nObjs.append(ChordEvaluation(fSets[h][i], chdAnnot[i], evalCols[3], window=window, delay=delay))
        
        # quality performance
        qObjs.append(ChordEvaluation(fSets[h][i], chdAnnot[i], evalCols[4], window=window, delay=delay))
        
        # mad performance
        madObjs.append(ChordEvaluation(fSets[h][i], chdAnnot[i], evalCols[5], window=window, delay=delay))
        
        # m21 performance
        m21Objs.append(ChordEvaluation(fSets[h][i], chdAnnot[i], evalCols[6], window=window, delay=delay))
        
        
    print('**********', fSetTtl[h])
    print('*** Pitch Class:')
    print(ChordSumEvaluation(pcObjs, name=None).tostring())
    print
    print('*** Pitch:')
    print(ChordSumEvaluation(pObjs, name=None).tostring())
    print
    print('*** Octave:')
    print(ChordSumEvaluation(oObjs, name=None).tostring())
    print
    print('*** Note:')
    print(ChordSumEvaluation(nObjs, name=None).tostring())
    print
    print('*** Quality:')
    print(ChordSumEvaluation(qObjs, name=None).tostring())
    print
    print('*** Mad Class:')
    print(ChordSumEvaluation(madObjs, name=None).tostring())
    print
    print('*** M21 Class:')
    print(ChordSumEvaluation(m21Objs, name=None).tostring())
    print
    print
    
    '''note: for cnn / dnn chord rec, precision is the key metric i.e., when I predict, do I predict correctly.'''

('**********', 'Note RNN Chord Pred')
*** Pitch Class:
sum for 30 files
  chords: 13500 TP: 10812 FP: 1397 FN: 2688 Precision: 0.886 Recall: 0.801 F-measure: 0.841 Acc: 0.726 mean:  13.5 ms std:   4.9 ms

*** Pitch:
sum for 30 files
  chords: 13500 TP: 10812 FP: 1397 FN: 2688 Precision: 0.886 Recall: 0.801 F-measure: 0.841 Acc: 0.726 mean:  13.5 ms std:   4.9 ms

*** Octave:
sum for 30 files
  chords: 13500 TP: 10261 FP: 1948 FN: 3239 Precision: 0.840 Recall: 0.760 F-measure: 0.798 Acc: 0.664 mean:  13.6 ms std:   5.0 ms

*** Note:
sum for 30 files
  chords: 13500 TP: 10034 FP: 2175 FN: 3466 Precision: 0.822 Recall: 0.743 F-measure: 0.781 Acc: 0.640 mean:  13.6 ms std:   4.9 ms

*** Quality:
sum for 30 files
  chords: 13500 TP: 10728 FP: 1481 FN: 2772 Precision: 0.879 Recall: 0.795 F-measure: 0.835 Acc: 0.716 mean:  13.5 ms std:   4.9 ms

*** Mad Class:
sum for 30 files
  chords: 13500 TP: 10997 FP: 1212 FN: 2503 Precision: 0.901 Recall: 0.815 F-measure: 0.855 Acc: 0.747 mean:  13.5 ms

/Users/mdowns/.virtualenvs/audio_2.7_env/lib/python2.7/site-packages/ipykernel/__main__.py:323: UserWarning: mean_error is given for all chords, this will change!
/Users/mdowns/.virtualenvs/audio_2.7_env/lib/python2.7/site-packages/ipykernel/__main__.py:332: UserWarning: std_error is given for all chords, this will change!


('**********', 'CNN Chord Preds')
*** Pitch Class:
sum for 30 files
  chords: 13500 TP:  1289 FP: 1629 FN: 12211 Precision: 0.442 Recall: 0.095 F-measure: 0.157 Acc: 0.085 mean:  50.1 ms std: 218.2 ms

*** Pitch:
sum for 30 files
  chords: 13500 TP:  1241 FP: 1677 FN: 12259 Precision: 0.425 Recall: 0.092 F-measure: 0.151 Acc: 0.082 mean:  47.2 ms std: 212.7 ms

*** Octave:
sum for 30 files
  chords: 13500 TP:     0 FP: 2918 FN: 13500 Precision: 0.000 Recall: 0.000 F-measure: 0.000 Acc: 0.000 mean:   nan ms std:   nan ms

*** Note:
sum for 30 files
  chords: 13500 TP:     0 FP: 2918 FN: 13500 Precision: 0.000 Recall: 0.000 F-measure: 0.000 Acc: 0.000 mean:   nan ms std:   nan ms

*** Quality:
sum for 30 files
  chords: 13500 TP:  1354 FP: 1564 FN: 12146 Precision: 0.464 Recall: 0.100 F-measure: 0.165 Acc: 0.090 mean:  66.1 ms std: 232.8 ms

*** Mad Class:
sum for 30 files
  chords: 13500 TP:   642 FP: 2276 FN: 12858 Precision: 0.220 Recall: 0.048 F-measure: 0.078 Acc: 0.041 mean:  21.0 

## 

In [323]:
a = [1,2,3]
b = ["a","b","c"]
pd.DataFrame({'a': a, 'b':b})



,a,b
0,1,a
1,2,b
2,3,c


### d. Common Functions

In [461]:
def save_chords(savTo, inDF):
    np.savetxt(savTo, inDF,
               fmt=['%.3f', '%.3f', '%.0f', '%s', '%.0f', '%s', '%s', '%s', '%s'], delimiter='\t')
    
def load_chords(inFname):   
    names = ['OnsetTime', 'OffsetTime', 'm21RootPitchClass', 
             'm21RootPitchAccidental', 'm21RootOctave', 
             'm21ContainsTriad', 'm21ChordQuality', 
             'm21PitchedCommonName', 'madChordLabel']
    
    return pd.read_table(inFname, names = names)

# test:
# tmp = load_chords('data/wip/AkPnBcht/MUS/MAPS_MUS-ty_mai_AkPnBcht.chords.dnn.txt')

def save_list(toSave, toDir, toName, toType):
    fullName = ''.join([toDir, toName, toType])
    with open(fullName, 'w') as f:
        for item in toSave:
            f.write(item + '\n')

def pickle_it(toPick, toDir, toName):
    with open(toDir + toName, 'wb') as f:
        pickle.dump(toPick, f)
        print("pickled:", toName)

def unpickle_it(frmDir, frmName):
    with open(frmDir + frmName, 'rb') as f:
        return(pickle.load(f))

In [462]:
#NEW_CHORD_DTYPE = [('OnsetTime', '%.3f'), ('OffsetTime', '%.3f'), 
#                       ('m21RootPitchClass', np.int), ('m21RootPitchAccidental', np.str),
#                       ('m21RootOctave', np.int), ('m21ContainsTriad', np.bool),
#                       ('m21ChordQuality', np.str),('m21PitchedCommonName', np.str),
#                       ('madChordLabel', np.str)] # 'U32'



In [ ]:
# put in "format" or "transform" function

# txt_to_y format: [OnsetTime, OffsetTime, Midipitch]

def flip_formats(inFile, frmFrmt, toFrmt):
    
    # valid to/froms: madNote, mapNote, madChord, m21Chord
    
    inHt, inWd = inFile.shape
    
    # data types
    MAD_NOTE_HEADER_1 = ['note_time', 'MIDI_note']
    MAD_NOTE_HEADER_2 = ['note_time', 'MIDI_note', 'duration']
    MAD_NOTE_HEADER_3 = ['note_time', 'MIDI_note', 'duration', 'MIDI_note']
        
    MAD_NOTE_DTYPE_1 = [('note_time', np.float), ('MIDI_note', np.int)]
    MAD_NOTE_DTYPE_2 = [('note_time', np.float), ('MIDI_note', np.int), ('duration', np.float)]
    MAD_NOTE_DTYPE_3 = [('note_time', np.float), ('MIDI_note', np.int), ('duration', np.float), ('MIDI_note', np.int)]
    
    MAD_CHORD_DTYPE = [('start', np.float), ('end', np.float), ('label', 'U32')]

    MAP_NOTE_HEADER = ['OnsetTime', 'OffsetTime', 'MidiPitch']
    MAP_CHORD_HEADER = ['OnsetTime', 'OffsetTime', 'ChordLabel']
    
    #M21_NOTE_DTYPE = 'f4,f4,int'
    #M21_CHORD_DTYPE = 'f4,f4,S10'
    
    if frmFrmt == "madNote":
        
        if toFrmt == "mapNote" or toFrmt == "m21Chord" or toFrmt == "madChord":
            
            notes = np.zeros(shape=(inHt, 3), dtype=float) # dtype defaults to float64
            notes[:,0] = inFile[:,0]
            notes[:,2] = inFile[:,1].astype(int)
            
            # if no duration
            if inFile.shape[1] == 2:
                notes[:,1] = 0
                
            # if duration
            else:
                notes[:,1] = inFile[:,0] + inFile[:,2]
                
            if toFrmt == "mapNote":
                return(notes)
        
            elif toFrmt == "m21Chord" or toFrmt == "madChord":
                
                notes = pd.DataFrame(notes, columns=['OnsetTime', 'OffsetTime', 'MidiPitch'])
                notes = tmp.round({'OnsetTime': 2, 'OffsetTime': 2, 'MidiPitch': 0})
                notes = tmp.sort_values(['OnsetTime', 'MidiPitch'],
                                      axis=0, ascending=True, inplace=False,
                                      kind='quicksort', na_position='last')
                
                notes["MidiPitch"] = tmp['MidiPitch'].astype(int)
                
                notes, chords = txt_to_y(tmp, mode="thisFile")
                
                if toFrmt == "m21Chord":
                    return(chords)
            
                elif toFrmt == "madChord":
                    # NOTE: UNTIL YOU FIND OUT HOW THEY'RE CLASSING THEIR CHORDS, DON'T SPEND TIME HERE.
                    lines = [line.rstrip('\n').split('\t') for line in open('data/mad2m21map.txt', 'U')]
                    headers = lines[0]; lines = lines[1:len(lines)]
                    lines = pd.DataFrame(lines, columns= headers)
                    
                    d = dict(zip(lines.m21_chord_pitchedCommonName, lines.mad_chord))
                    
                    for i in range(chords.shape[0]):
                        try:
                            chords.ChordLabel.iloc[i] = d[chords.ChordLabel.iloc[i]]
                        except:
                            chords.ChordLabel.iloc[i] = 'N'
                            
                    return(chords)
        # NOTE: DON'T SPEND TIME GOING FROM MAD.CHORD LABELS TO M21. JUST USE MODEL OUTPUTS TO PREDICT. 
                
                

#print(flip_formats(rnn_note_detect, "madNote", "mapNote"))
print(flip_formats(rnn_note_detect, "madNote", "m21Chord"))
print(flip_formats(rnn_note_detect, "madNote", "madChord"))


## 2. Train Model

### a. process batches

### b. evaluate results

# Output Results

sonic visualizer: music analysis http://www.sonicvisualiser.org/
vamp plugins: http://www.vamp-plugins.org/download.html
bbc plugins: human/music, intensity, energy https://github.com/bbcrd/bbc-vamp-plugins/blob/master/README.md
chordino: maj/min chord recognition http://www.isophonics.net/nnls-chroma
music matching: http://www.eecs.qmul.ac.uk/~simond/match/index.html

building plugins: http://www.vamp-plugins.org/develop.html
annotations: http://www.vamp-plugins.org/sonic-annotator/

musescore: score notation from midi, music xml https://github.com/musescore/MuseScore

## 1. midi-note

## 2. midi-chord

# Utilities

## 1. Compose midi

# Read

1. Files from directory, OR
2. Stream

# V1: Import piano train / test data

1. Take their lists of train and test data
2. Create list object
3. Feed it to a process to either iteratively or bulk load files from directory
4. Perform log scale transform of input wav
5. Come back to other features, chords


# Learn to map sound to notes

1. Take professional music sound files
2. Play, logging spectrum (frequency / time) and other attributes
    a. https://github.com/tyiannak/pyAudioAnalysis/wiki/3.-Feature-Extraction, OR
    b. 
3. Predict notes based on sound
    a. input spectrum is the "X"
    b. sheet music notes are the "y" (notes A, B, C, etc.)

# V1: Generate harp note / chord train / test files

Garageband or other to generate:
1. individual instrument note (pitch?), chord by major/minor, octave, inversion
2. sequence files w/ varying amounts of spacing
3. mp3's of classical music for which you can easily veryify the notes


# V2: Standard: Fingerprint music files

## n. Fingerprint music

https://github.com/dpwe/audfprint

Audfprint is a python (and Matlab) script that can take a list of soundfiles and create a database of landmarks, and then subsequently take one or more query audio files and match them against the previously-created database.  The fingerprint is robust to things like time skews, different encoding schemes, and even added noise. It can match small fragments of sound, down to 10 sec or less.

In [ ]:
# use FFT, MFCC, etc to create a fignerprint of each of the music files s/t when user starts playing, you can take notes they've played and match 

# V2: Streaming: Extract note Sample

# V2: Streaming: Search / match sample to fingerprints

# Establish stream

Use pyaudio to instantiate stream for practice session

# Display mode

1. Accept song selection
2. Load / display sheet music
3. Listen for start
4. Recieve sounds / translate notes
5. Track progress w/ vertical bar
6. Spot repeats re-setting tracking bar

In [ ]:
# https://github.com/tyiannak/pyAudioAnalysis
# http://essentia.upf.edu/documentation/


# Evaluate mode

1. display mode functionality including tracking progress
2. record playing. overlay repeats. you're tracking stats. so, obj s/b:
    a. to get through song re-playing pieces as required, THEN
    b. to get through song cleanly
3. comparing played to professional
    a. option to play:
        i. metrinome 
        ii. professional a low volume
4. identify discrpancies (timing after prior note, incorrect note)
    a. Gaia, a C++ library with python bindings which implement similarity measures and classification on the results of audio analysis, and generate classification models that Essentia can use to compute high-level description of music.
5. show discrepancies
    a. accept tolerances (+/- time, other?)
    b. show played note in red (i.e., before/after, above/below).
6. show / log statistics
    a. accuracy
    b. similarity
    c. error types and frequency distribution
        i. early,
        ii. late
        iii. wrong note
    d. problem areas

# Interactive mode

1. evaluate mode functionality
2. prompt session info and imprint voice of user for command interface
    a. "this is []. the date is []. i'll be practicing for about [] minutes."
3. voice commands
    a. "replay [] notes" - defaults to: 5 notes, played version
    b. "replay base [] notes"
    c. "loop [] notes" - 
        ii. Loop [] notes / Stop Loop
2. 

# References

This project would not be possible without the invaluable assistance of:

## Training data

The MAPS piano data set. Roughly 40G of piano notes, chords, music assembled by V. Emiya for her PhD thesis at Telecom ParisTech/ENST in 2008 and in conjunction with R. Badeau, B. David for their paper "Multipitch estimation of piano sounds using a new probabilistic spectral smoothness principle"<cite data-cite="emiya2010multipitch"></cite>


In [ ]:
# for installing latex, bibtex and pdf-ing jupyter notenooks: https://www.youtube.com/watch?v=m3o1KXA1Rjk